In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import utility
import math
from time import time
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from sklearn.metrics import classification_report

In [2]:
def load_column():
    df = pd.read_csv('./datasets/column_2C.dat', sep=" ", names=['pelvic_incidence', 'pelvic_tilt', 'lumbar_lordosis_angle', 'sacral_slope', 'pelvic_radius', 'degree_spondylolisthesis','target'])
    df['target']=np.where(df['target']=='AB',1,0)
    return df

def load_blood_transfusion():
    df = pd.read_csv('./datasets/blood_transfusion.csv')
    return df

def load_ionosphere():
    df = pd.read_csv('./datasets/Ionosphere.csv')
    return df

def load_parkinson():
    df = pd.read_csv('./datasets/parkinsons.csv')
    return df

def load_pima():
    df = pd.read_csv('./datasets/diabetes.csv')
    df.rename(columns={'Outcome': 'target'}, inplace=True)
    return df

In [3]:
#SklearnDatasets
df = load_pima()
dir_path = 'Pima'

In [4]:
scaler = MinMaxScaler()
scaler.fit(df.values[:, :-1])
scaled_df = scaler.transform(df.values[:, :-1])
lower_bound = scaled_df.min()
upper_bound = scaled_df.max()
print(lower_bound, upper_bound)

0.0 1.0


In [5]:
df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
df_scaled['target'] = targets
num_classes = len(df_scaled['target'].unique())
display(df_scaled)

Original Targets:  [0. 1.] 
Desired Targets: [0,1]
Is original the desired [0, 1]?  True


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,target
0,0.352941,0.743719,0.590164,0.353535,0.000000,0.500745,0.234415,0.483333,1
1,0.058824,0.427136,0.540984,0.292929,0.000000,0.396423,0.116567,0.166667,0
2,0.470588,0.919598,0.524590,0.000000,0.000000,0.347243,0.253629,0.183333,1
3,0.058824,0.447236,0.540984,0.232323,0.111111,0.418778,0.038002,0.000000,0
4,0.000000,0.688442,0.327869,0.353535,0.198582,0.642325,0.943638,0.200000,1
...,...,...,...,...,...,...,...,...,...
763,0.588235,0.507538,0.622951,0.484848,0.212766,0.490313,0.039710,0.700000,0
764,0.117647,0.613065,0.573770,0.272727,0.000000,0.548435,0.111870,0.100000,0
765,0.294118,0.608040,0.590164,0.232323,0.132388,0.390462,0.071307,0.150000,0
766,0.058824,0.633166,0.491803,0.000000,0.000000,0.448584,0.115713,0.433333,1


In [6]:
np.random.seed(50)
X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
n_neurons = X_train.shape[1]
n_hidden_layers = 1
print(f'n_hidden_layers: {n_hidden_layers}, n_neurons {n_neurons}')
model = tf.keras.Sequential([
    tf.keras.Input(shape=(X_train.shape[1],)),
    layers.Dense(n_neurons, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model_path = f'new_models/{dir_path}.keras'
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=40)
ck = tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_accuracy', save_best_only=True)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)
start = time()
model.fit(X_train,y_train, epochs=400, batch_size=32,validation_data=(X_test, y_test), verbose=1, callbacks=[ck, es])
print(f'Training time: {time()-start}')
model = tf.keras.models.load_model(model_path)
print('Training Results')
model.evaluate(X_train, y_train, verbose=2)
print('Test Results')
model.evaluate(X_test, y_test, verbose=2)
model.save(f'new_models/{dir_path}.h5', save_format='h5')

n_hidden_layers: 1, n_neurons 8


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 8)                   │              72 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │              18 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 90 (360.00 B)

 Trainable params: 90 (360.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/400
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.6498 - loss: 0.6698 - val_accuracy: 0.6510 - val_loss: 0.6743
Epoch 2/400
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6501 - loss: 0.6628 - val_accuracy: 0.6510 - val_loss: 0.6731
Epoch 3/400
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6437 - loss: 0.6707 - val_accuracy: 0.6510 - val_loss: 0.6720
Epoch 4/400
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6359 - loss: 0.6683 - val_accuracy: 0.6510 - val_loss: 0.6711
Epoch 5/400
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6815 - loss: 0.6501 - val_accuracy: 0.6510 - val_loss: 0.6700
Epoch 6/400
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6594 - loss: 0.6584 - val_accuracy: 0.6510 - val_loss: 0.6691
Epoch 7/400
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6347 - loss: 0.6648 - val_accuracy: 0.6510 - val_loss: 0.6683
Epoch 8/400
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6192 - loss: 0.6729 - val_accuracy: 0.6510 - val_

In [7]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  

y_test_classes = np.argmax(y_test, axis=1)

report = classification_report(y_test_classes, y_pred_classes,digits=4)
print(report)

# Save the report to a text file
with open(f'new_models/{dir_path}_report.txt', 'w') as f:
    f.write(report)

18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
              precision    recall  f1-score   support

           0     0.7678    0.8640    0.8130       375
           1     0.6688    0.5124    0.5803       201

    accuracy                         0.7413       576
   macro avg     0.7183    0.6882    0.6967       576
weighted avg     0.7332    0.7413    0.7318       576

